# The aHB circuit tracks fictive trajectories

We have seen in the previous notebooks that the aHB integrator could be capabable of keep track of past motion over a significant amount of time. In this notebook, we will see how much it can keep track of past trajectories over significant amounts of time.

## A first look at cumulative directional motion and network phase

In [ ]:
%matplotlib widget
from pathlib import Path

import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from bouter.utilities import crop
from lotr import A_FISH, LotrExperiment, dataset_folders
from lotr.behavior import get_bouts_props_array
from lotr.default_vals import REGRESSOR_TAU_S
from lotr.utils import convolve_with_tau
from matplotlib import pyplot as plt
from scipy.stats import spearmanr
from tqdm import tqdm

COLS = pltltr.COLS

In [ ]:
# First, get the unwrapped phase:
exp = LotrExperiment(A_FISH)
network_phase = np.unwrap(exp.network_phase)

# Now, let's find a way of reconstruct past motion of the fish.
# We will start with a vector of zeros that has ones at timepoints
# where the fish swam:
theta_turned = get_bouts_props_array(
    exp.n_pts, exp.bouts_df, selection="all", value="bias"
)

# Fictive heading will be the cumulative sum of this array:
fictive_head = np.cumsum(theta_turned)

# Finally, we smooth this array with a kernel that matches the slower dynamics
# of the neuron (results are not much affected by this)
fictive_head = convolve_with_tau(np.cumsum(theta_turned), REGRESSOR_TAU_S * exp.fn)

In [ ]:
f, axs = plt.subplots(
    3, 1, figsize=(5, 3), sharex=True, gridspec_kw=(dict(bottom=0.12))
)
lw = 1

labels = ["Tail angle", "Angle turned (dθ)", "Fict. heading (θ)"]
col_seq = [COLS["beh"], pltltr.shift_lum(COLS["th_plot"], 0.2), COLS["th_plot"]]
for i, (x, y) in enumerate(
    [
        (exp.behavior_log["t"], exp.behavior_log["tail_sum"]),
        (exp.time_arr, theta_turned),
        (exp.time_arr, fictive_head),
    ]
):
    axs[i].plot(
        x, y, lw=lw, c=col_seq[i], label=labels[i], rasterized=i == 0
    )  # rasterize for exporting
    axs[i].legend(loc=1, bbox_to_anchor=(1.1, 1.1))

axs[0].set(**pltltr.get_pi_labels(1, ax="y"))
axs[1].set(**pltltr.get_pi_labels(0.5, ax="y"), ylim=(-1.8, 1.8))
axs[2].set(xlabel="Time (s)", **pltltr.get_pi_labels(coefs=(0, -5, -10, -15), ax="y"))

[pltltr.despine(ax) for ax in axs.flatten()]
plt.show()

Now, how does the network phase look, compared with the fictive heading?

In [ ]:
f, ax = plt.subplots(
    1, 1, figsize=(4, 1.5), gridspec_kw=(dict(bottom=0.25, right=0.85))
)
twin_ax = ax.twinx()
ax.plot(
    exp.time_arr, fictive_head, c=COLS["th_plot"],
)
twin_ax.plot(
    exp.time_arr, np.unwrap(exp.network_phase), c=COLS["ph_plot"],
)
ax.set_ylabel("Fict. heading (θ)", c=COLS["th_plot"])
ax.set(xlabel="Times (s)", **pltltr.get_pi_labels(coefs=(0, -5, -10, -15), ax="y"))
twin_ax.set_ylabel("Network phase (ϕ)", c=COLS["ph_plot"])
twin_ax.set(
    ylim=twin_ax.get_ylim()[::-1], **pltltr.get_pi_labels(coefs=(0, 2, 4, 6), ax="y")
)
rho, pval = spearmanr(fictive_head, np.unwrap(exp.network_phase))
ax.text(1200, -np.pi, "$ρ_r=" + f"{rho:0.3f}" + "$" + pltltr.get_pval_stars(pval))
[pltltr.despine(a, ["top"]) for a in [ax, twin_ax]]